In [33]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen, urlretrieve
import requests
import subprocess
import threading
import re
import os

Completed fetching record g19960421
Unpacking...
Done!


In [2]:
NEAT_vol_1 = 'https://sbnarchive.psi.edu/pds3/neat/geodss/data/'
NEAT_vol_1_folder = "geodss"
NEAT_vol_2 = 'https://sbnarchive.psi.edu/pds3/neat/tricam/data/'
NEAT_vol_2_folder = "tricam"
NEAT_vol_3 = 'https://sbnarchive.psi.edu/pds3/neat/tricam2/data/'
NEAT_vol_3_folder = "tricam2"

In [3]:
def pullVolumeData(url, ext=''):
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    links = [url + '/' + node.get('href') for node in soup.find_all('a') if node.get('href').endswith(ext)]
    return links[5:-2]

In [4]:
NEAT_vol_1_data = pullVolumeData(NEAT_vol_1)

In [5]:
def getRecordsFromVolumeData(data, _idx):
    if isinstance(_idx, int):
        return [data[_idx]]
    else:
        x = []
        for i in _idx:
            x.append(data[i])
        return x

In [6]:
nv1s1 = getRecordsFromVolumeData(NEAT_vol_1_data, [1])
nv1s1

['https://sbnarchive.psi.edu/pds3/neat/geodss/data//g19960418/']

In [30]:
def read_url(url,rL):
    url = url.replace(" ","%20")
    req = Request(url)
    a = urlopen(req).read()
    soup = BeautifulSoup(a, 'html.parser')
    x = (soup.find_all('a'))
    for i in x:
        file_name = i.extract().get_text()
        url_new = url + file_name
        url_new = url_new.replace(" ","%20")
        if(file_name[-1]=='/' and file_name[0]!='.'):
            read_url(url_new,rL)
        rL.append(url_new)

def getRecordID(record):
    return record.split('/')[-2]
        
def fetchRecord(record,verbose=False):
    file_idx = 0
    recordData=[]
    read_url(record,recordData)
    print("Fetching record " + getRecordID(record) + "...")
    # print("wget -r -nH -nc --cut-dirs=2 --no-parent --reject=\"index.html*\" " + str(record))
    os.system("wget -r -nH -nc --cut-dirs=2 --no-parent --reject=\"index.html*\" " + str(record))
    if(verbose):
        print("Completed fetching record " + getRecordID(record) , end="\r")
    else:
        print("Completed fetching record " + getRecordID(record))
    print("Unpacking...")
    subprocess.call("funpacker.sh", shell=True)
    print("Done!")
    
def fetchRecords(data):
    for record in data:
        t = threading.Thread(target=fetchRecord, args=(record,False));
        t.daemon = True;
        t.start();

In [31]:
_idx=[2,3,4]
geodss_samples = getRecordsFromVolumeData(NEAT_vol_1_data, _idx);
geodss_samples

['https://sbnarchive.psi.edu/pds3/neat/geodss/data//g19960419/',
 'https://sbnarchive.psi.edu/pds3/neat/geodss/data//g19960420/',
 'https://sbnarchive.psi.edu/pds3/neat/geodss/data//g19960421/']

In [32]:
fetchRecords(geodss_samples)

Fetching record g19960420...
Fetching record g19960419...
Fetching record g19960421...
Completed fetching record g19960420
Unpacking...


Exception in thread Thread-14:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-30-0c6454a5e08a>", line 30, in fetchRecord
    subprocess.call("funpacker.sh", shell=True)
NameError: name 'subprocess' is not defined



Completed fetching record g19960419
Unpacking...


Exception in thread Thread-13:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-30-0c6454a5e08a>", line 30, in fetchRecord
    subprocess.call("funpacker.sh", shell=True)
NameError: name 'subprocess' is not defined

